In [139]:
import pandas as pd
import os
import re
import numpy as np
from collections import Counter
from gensim.models import FastText
import jieba
import networkx as nx
import math
from gensim.models.word2vec import LineSentence
from sklearn.metrics.pairwise import cosine_similarity
from functools import partial

In [2]:
news_data = pd.read_csv('../input/sqlResult_1558435.csv', encoding='gb18030')
news_data.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [3]:
# with open('corpus.txt', 'w', encoding='utf-8') as fout:
#     for sent in news_data['content']:
#         if pd.isna(sent):
#             continue
#         sent = re.sub(r'[\r\n]', '', sent)
#         wl = jieba.lcut(sent)
#         if wl:
#             fout.write(' '.join(wl))
#             fout.write('\n')

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\King\AppData\Local\Temp\jieba.cache
Loading model cost 0.810 seconds.
Prefix dict has been built succesfully.


In [31]:
len_arr = np.array([len(news_data['content'][i]) for i in range(100)])

In [36]:
len_arr

array([ 177,  329,  216,   45, 1674, 1056,  657, 4093, 3665,  755,  169,
       1467,  487,  774, 1137,  331,  293,  495,  289, 1707,  928, 2453,
        761,  531,  423,  129,  678, 1536,  638, 3478, 1150, 1738,  136,
        501,  242,  400,  327,  321,  523,   45, 1736,  662, 1133,  899,
        241,  147, 2369, 1536, 3364,  596,  140,  161, 2520,  415, 1312,
        376,  398,  749,  941,  359,  607, 1733,  625, 1248,  607, 1821,
        830,  950,  960,  543, 1643, 1519,  527,  221,  109,  201,  210,
        782,  622,  209,  999,  394,  246, 1049,  591,  620, 1032,  182,
         69,  223, 2815, 2086,  289,  761,  584, 1153,  841, 5572,  928,
        287])

In [46]:
news_data['content'][43]

'上周周末，中超第13轮已经全部战罢，其中，广州恒大战胜贵州恒丰智诚，豪取联赛九连胜的同时继续领跑积分榜，队内的曾诚与张琳芃也入选了本轮中超的最佳阵容。\r\n门将：曾诚。尽管在国家队表现不佳，但回到俱乐部的曾诚仅用了一场比赛就为自己实现了正名。全场5次扑救则次次关键，而且，曾诚还扑出了耶拉维奇可能扳平比分的点球，力助恒大豪取九连胜。\r\n后卫：\r\n张琳芃，攻防俱佳的张莫斯，此役在防守端贡献6次解围、3次拦截和2次抢断。在进攻端，张琳芃则有4次传中和1次突破，并直接助攻了郜林打破僵局的进球。\r\n戈麦斯，这一轮他共做出了8次解围、3次拦截、1次抢断，作为队长出战的戈麦斯继续扮演着建业铁闸的角色。\r\n伊斯梅洛夫，本场面对气势汹汹的江苏苏宁，伊斯梅洛夫在防守端6次解围、3次抢断，此外还有3次封堵射门和1次拦截，是亚泰客场告捷的幕后英雄。\r\n李磊，京津德比中李磊送出5次抢断、3次拦截和1次解围的全面数据，而且他还助攻伊尔马兹取得进球。\r\n中场：\r\n姆比亚，本轮比赛，姆比亚在与权健中场核心维特塞尔的对抗中丝毫不落下风，最后时刻还用一记制胜球助华夏幸福继续取胜。\r\n张晓彬，出任后腰的张晓彬做出5次抢断、4次拦截，传球成功率超过90%，他在中场全面的表现也在赛后得到了好评。\r\n多奇卡尔,此役，多奇卡尔先是造成了对手的乌龙球，随后又巧妙助攻哈维尔破门锁定胜局，可以说是建业取胜的最大功臣之一。\r\n前锋：\r\n曹赟定，全场完成4次射门，并送出3次威胁传球，此外还有惊人的8次突破和12次传中，几乎凭借一己之力搅的力帆禁区天翻地覆。\r\n卡尔德克，跟曹赟定一样，本场比赛都是一传一射，而且卡尔德克的进攻桥头堡作用非常关键。\r\n塔尔德利，继上次出场梅开二度后，塔尔德利又在本轮独中两元。而且比赛中，“塔神”5次射门全部射正，向前传球居多的他，传球成功率竟然高达86%，毫无疑问，本赛季脱胎换骨的塔神已经成为了鲁能进攻中的大腿。\r\n此外，河南建业的新任主帅亚森被评选为本轮最佳教练。上任不过三天，亚森就帮助河南建业拿到了本赛季的第二场胜利，换帅如换刀，亚森可谓是救火教练的典范。\r\n你心目中，本轮的最佳球员是谁？\r\n'

In [165]:
news = news_data['content'][43]

In [110]:
def split_sentence(sent: str):
    sent = re.sub(r'[\r\n]', '', sent)
    ls = re.split('([，。？！,.])', sent)
    pat = '，。？！,.'
    sent_ids = []
    symbol_ids = []
    for i, s in enumerate(ls):
        if s in pat:
            symbol_ids.append(i)
        else:
            sent_ids.append(i)
    return ls, sent_ids, symbol_ids

#     st = []
#     ls = re.split('，', sent)
#     for i in range(len(ls)-1):
#         ls[i] += '，'
    
#     for s in ls:
#         lls = re.split('。', s)
#         for i in range(len(lls)-1):
#             lls[i] += '。'
#         st += lls
#     ret = []
#     for s in st:
#         ls = re.split(' ', s)
#         ret += ls
#     return ret
    # return re.split(r'[，。,. ？！]', sent)

In [7]:
# def split_sentence(sentence):
#     pattern = re.compile('[。，,.]：')
#     split = pattern.sub(' ', sentence).split()  # split sentence
#     return split

In [106]:
re.split('([，。？！,. ])', news)

['上周周末',
 '，',
 '中超第13轮已经全部战罢',
 '，',
 '其中',
 '，',
 '广州恒大战胜贵州恒丰智诚',
 '，',
 '豪取联赛九连胜的同时继续领跑积分榜',
 '，',
 '队内的曾诚与张琳芃也入选了本轮中超的最佳阵容',
 '。',
 '\r\n门将：曾诚',
 '。',
 '尽管在国家队表现不佳',
 '，',
 '但回到俱乐部的曾诚仅用了一场比赛就为自己实现了正名',
 '。',
 '全场5次扑救则次次关键',
 '，',
 '而且',
 '，',
 '曾诚还扑出了耶拉维奇可能扳平比分的点球',
 '，',
 '力助恒大豪取九连胜',
 '。',
 '\r\n后卫：\r\n张琳芃',
 '，',
 '攻防俱佳的张莫斯',
 '，',
 '此役在防守端贡献6次解围、3次拦截和2次抢断',
 '。',
 '在进攻端',
 '，',
 '张琳芃则有4次传中和1次突破',
 '，',
 '并直接助攻了郜林打破僵局的进球',
 '。',
 '\r\n戈麦斯',
 '，',
 '这一轮他共做出了8次解围、3次拦截、1次抢断',
 '，',
 '作为队长出战的戈麦斯继续扮演着建业铁闸的角色',
 '。',
 '\r\n伊斯梅洛夫',
 '，',
 '本场面对气势汹汹的江苏苏宁',
 '，',
 '伊斯梅洛夫在防守端6次解围、3次抢断',
 '，',
 '此外还有3次封堵射门和1次拦截',
 '，',
 '是亚泰客场告捷的幕后英雄',
 '。',
 '\r\n李磊',
 '，',
 '京津德比中李磊送出5次抢断、3次拦截和1次解围的全面数据',
 '，',
 '而且他还助攻伊尔马兹取得进球',
 '。',
 '\r\n中场：\r\n姆比亚',
 '，',
 '本轮比赛',
 '，',
 '姆比亚在与权健中场核心维特塞尔的对抗中丝毫不落下风',
 '，',
 '最后时刻还用一记制胜球助华夏幸福继续取胜',
 '。',
 '\r\n张晓彬',
 '，',
 '出任后腰的张晓彬做出5次抢断、4次拦截',
 '，',
 '传球成功率超过90%',
 '，',
 '他在中场全面的表现也在赛后得到了好评',
 '。',
 '\r\n多奇卡尔',
 ',',
 '此役',
 '，',
 '多奇卡尔先是造成了对手的乌龙球',
 '，',
 '随后又巧妙助攻哈

In [108]:
split_sentence(news)

(['上周周末，中超第13轮已经全部战罢，其中，广州恒大战胜贵州恒丰智诚，豪取联赛九连胜的同时继续领跑积分榜，队内的曾诚与张琳芃也入选了本轮中超的最佳阵容',
  '。',
  '门将：曾诚',
  '。',
  '尽管在国家队表现不佳，但回到俱乐部的曾诚仅用了一场比赛就为自己实现了正名',
  '。',
  '全场5次扑救则次次关键，而且，曾诚还扑出了耶拉维奇可能扳平比分的点球，力助恒大豪取九连胜',
  '。',
  '后卫：张琳芃，攻防俱佳的张莫斯，此役在防守端贡献6次解围、3次拦截和2次抢断',
  '。',
  '在进攻端，张琳芃则有4次传中和1次突破，并直接助攻了郜林打破僵局的进球',
  '。',
  '戈麦斯，这一轮他共做出了8次解围、3次拦截、1次抢断，作为队长出战的戈麦斯继续扮演着建业铁闸的角色',
  '。',
  '伊斯梅洛夫，本场面对气势汹汹的江苏苏宁，伊斯梅洛夫在防守端6次解围、3次抢断，此外还有3次封堵射门和1次拦截，是亚泰客场告捷的幕后英雄',
  '。',
  '李磊，京津德比中李磊送出5次抢断、3次拦截和1次解围的全面数据，而且他还助攻伊尔马兹取得进球',
  '。',
  '中场：姆比亚，本轮比赛，姆比亚在与权健中场核心维特塞尔的对抗中丝毫不落下风，最后时刻还用一记制胜球助华夏幸福继续取胜',
  '。',
  '张晓彬，出任后腰的张晓彬做出5次抢断、4次拦截，传球成功率超过90%，他在中场全面的表现也在赛后得到了好评',
  '。',
  '多奇卡尔',
  ',',
  '此役，多奇卡尔先是造成了对手的乌龙球，随后又巧妙助攻哈维尔破门锁定胜局，可以说是建业取胜的最大功臣之一',
  '。',
  '前锋：曹赟定，全场完成4次射门，并送出3次威胁传球，此外还有惊人的8次突破和12次传中，几乎凭借一己之力搅的力帆禁区天翻地覆',
  '。',
  '卡尔德克，跟曹赟定一样，本场比赛都是一传一射，而且卡尔德克的进攻桥头堡作用非常关键',
  '。',
  '塔尔德利，继上次出场梅开二度后，塔尔德利又在本轮独中两元',
  '。',
  '而且比赛中，“塔神”5次射门全部射正，向前传球居多的他，传球成功率竟然高达86%，毫无疑问，本赛季脱胎换骨的塔神已经成为了鲁能进攻中的大腿',
  '。',
  '此外，河南建业的新任主帅亚森被评

In [120]:
def get_stop_words(file: str, encoding='utf-8'):
    ret = [l for l in open(file, encoding=encoding).read()]
    return ret

In [121]:
def clean_sentences(sents: list, stop_words: list):
    ret = {}
    for i, sent in enumerate(sents):
        words = jieba.lcut(sent)
        wds = []
        for w in words:
            if w not in stop_words:
                wds.append(w)
        if wds:
            ret[i] = wds
    return ret

In [122]:
def sentence_similarity_1(sent1, sent2):
    ret = 0
    same_cnt = 0
    for w in sent1:
        if w in sent2:
            same_cnt += 1
    ret = same_cnt / (math.log2(len(sent1)) + math.log2(len(sent2)) + 1)
    return ret

In [123]:
# Calculate sentence similarity by embedding vector
def sentence_similarity_func(model, prob_func):
    a = 0.001
    col = model.wv.vector_size

    def core(sent1, sent2):
        vec1 = np.zeros(col)
        vec2 = np.zeros(col)
        for w in sent1:
            pw = a / (a + prob_func(w))
            vec1 += pw * model.wv[w]
        for w in sent2:
            pw = a / (a + prob_func(w))
            vec2 += pw * model.wv[w]
        ret = 0
        ret = cosine_similarity(vec1.reshape(1,-1), vec2.reshape(1,-1))[0][0]
        return ret
    return core

In [130]:
def sentence_embedding(model, prob_func, stop_words):
    a = 0.001
    col = model.wv.vector_size
    
    def core(sent):
        vec = np.zeros(col)
        words = jieba.lcut(sent)
        for w in words:
            if w in model.wv.vocab and w not in stop_words:
                pw = a / (a + prob_func(w))
                vec += pw * model.wv[w]
        return vec
    return core

In [125]:
def get_prob(counter):
    total_cnt = sum(counter.values())
    def core(word: str):
        return counter[word] / total_cnt
    return core

In [9]:
model = FastText(LineSentence('corpus.txt'), window=5, size=35, iter=10, min_count=1)

In [ ]:
def gen_sim_mat(sent_dict: dict, sim_func):
    size = len(sent_dict)
    ret = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            if i != j:
                ret[i][j] = sim_func(sent_dict[i], sent_dict[j])
    return ret

In [ ]:
def get_final_sentence(sent_list, indices, word_cnt=200):
    cur_cnt = 0
    lst = []
    for i in indices:
        sent = sent_list[i]
        size = len(sent)
        cur_cnt += size
        if cur_cnt < word_cnt:
            lst.append(i)
    ret = '。'.join([sent_list[i] for i in lst])
    return ret

In [126]:
stop_words = get_stop_words('../input/stop_words.txt')

In [159]:
def get_correlations(sents, sent_vec_func):
    text = ' '.join(sents)
    text_vec = sent_vec_func(text)
    sims = []
    for sent in sents:
        vec = sent_vec_func(sent)
        sim = cosine_similarity(vec.reshape(1, -1), text_vec.reshape(1, -1))[0][0]
        sims.append(sim)
    ret = [(sents[ind], sims[ind]) for ind in sorted(range(len(sims)), key=lambda i: sims[i], reverse=True)]
    return ret

In [132]:
tokeners = []
for line in open('corpus.txt', 'r', encoding='utf-8'):
    tokeners += line.split()

In [133]:
word_counter = Counter(tokeners)

In [134]:
prob_func = get_prob(word_counter)

In [135]:
get_sentence_vec = sentence_embedding(model, prob_func, stop_words)

In [161]:
score_func_embed = partial(get_correlations, sent_vec_func=get_sentence_vec)

In [168]:
get_summarization_simple(news, score_func_embed, 200)

'曾诚还扑出了耶拉维奇可能扳平比分的点球，后卫：张琳芃，此役在防守端贡献6次解围、3次拦截和2次抢断。并直接助攻了郜林打破僵局的进球。而且他还助攻伊尔马兹取得进球。本轮比赛，姆比亚在与权健中场核心维特塞尔的对抗中丝毫不落下风，此役，随后又巧妙助攻哈维尔破门锁定胜局，全场完成4次射门，继上次出场梅开二度后，塔尔德利又在本轮独中两元。本赛季脱胎换骨的塔神已经成为了鲁能进攻中的大腿。亚森就帮助河南建业拿到了本赛季的第二场胜利，本轮的最佳球员是谁？'

In [136]:
model.wv['小米']

array([ 0.9608598 ,  1.9616965 , -0.6550164 , -0.08581933,  0.4027366 ,
       -0.49193567, -0.98731744, -1.1231472 , -2.2945056 , -2.0371764 ,
        0.48530376,  0.99433416,  2.6392832 , -1.3752371 , -1.2738191 ,
       -0.79540163, -0.5047238 , -0.8784493 ,  4.56962   , -2.9469717 ,
        0.36588308, -0.81705254, -1.0014058 , -0.4530072 , -2.7770162 ,
        0.30021167,  4.8232155 , -0.39225984, -0.9452708 , -1.5930241 ,
       -0.7761965 , -1.1576653 , -0.6784643 ,  0.48199782,  1.961022  ],
      dtype=float32)

In [137]:
get_sentence_vec('小米')

array([ 0.94877106,  1.93701601, -0.64677554, -0.08473962,  0.3976697 ,
       -0.48574653, -0.97489583, -1.10901678, -2.26563811, -2.01154637,
        0.47919807,  0.98182428,  2.60607791, -1.35793507, -1.25779295,
       -0.78539455, -0.49837378, -0.86739743,  4.51212883, -2.90989542,
        0.36127985, -0.80677307, -0.98880696, -0.44730783, -2.74207807,
        0.29643464,  4.76253366, -0.38732475, -0.93337816, -1.57298207,
       -0.76643103, -1.1431005 , -0.66992843,  0.47593373,  1.93635011])

In [93]:
def get_connect_graph_by_text_rank(tokenized_text, window=3):
    keywords_graph = nx.Graph()
    tokeners = tokenized_text
    print(tokeners)
    for ii, t in enumerate(tokeners):
        word_tuples = [(tokeners[connect], t) 
                       for connect in range(ii-window, ii+window+1) 
                       if connect >= 0 and connect < len(tokeners)]
        keywords_graph.add_edges_from(word_tuples)

    return keywords_graph

In [94]:
def get_summarization_simple_with_text_rank(text, constraint=200):
    return get_summarization_simple(text, sentence_ranking_by_text_ranking, constraint)

In [95]:
def sentence_ranking_by_text_ranking(split_sentence):
    sentence_graph = get_connect_graph_by_text_rank(split_sentence)
    ranking_sentence = nx.pagerank(sentence_graph)
    ranking_sentence = sorted(ranking_sentence.items(), key=lambda x: x[1], reverse=True)
    print(ranking_sentence)
    return ranking_sentence

In [96]:
def get_summarization_simple(text, score_fn, constraint=200):
    sents, sent_ids, symb_ids = split_sentence(text)
    sub_sentence = [sents[i] for i in sent_ids]
    ranking_sentence = score_fn(sub_sentence)
    selected_text = set()
    current_text = ''
    
    for sen, _ in ranking_sentence:
        if len(current_text) < constraint:
            current_text += sen
            selected_text.add(sen)
        else:
            break

    summarized = []
    for sen in sub_sentence:  # print the selected sentence by sequent
        if sen in selected_text:
            ind = sents.index(sen)
            summarized.append(sen)
            if (ind + 1) in symb_ids:
                summarized.append(sents[ind + 1])
    return ''.join(summarized)

In [167]:
get_summarization_simple_with_text_rank(news, 250)

['上周周末', '中超第13轮已经全部战罢', '其中', '广州恒大战胜贵州恒丰智诚', '豪取联赛九连胜的同时继续领跑积分榜', '队内的曾诚与张琳芃也入选了本轮中超的最佳阵容', '门将：曾诚', '尽管在国家队表现不佳', '但回到俱乐部的曾诚仅用了一场比赛就为自己实现了正名', '全场5次扑救则次次关键', '而且', '曾诚还扑出了耶拉维奇可能扳平比分的点球', '力助恒大豪取九连胜', '后卫：张琳芃', '攻防俱佳的张莫斯', '此役在防守端贡献6次解围、3次拦截和2次抢断', '在进攻端', '张琳芃则有4次传中和1次突破', '并直接助攻了郜林打破僵局的进球', '戈麦斯', '这一轮他共做出了8次解围、3次拦截、1次抢断', '作为队长出战的戈麦斯继续扮演着建业铁闸的角色', '伊斯梅洛夫', '本场面对气势汹汹的江苏苏宁', '伊斯梅洛夫在防守端6次解围、3次抢断', '此外还有3次封堵射门和1次拦截', '是亚泰客场告捷的幕后英雄', '李磊', '京津德比中李磊送出5次抢断、3次拦截和1次解围的全面数据', '而且他还助攻伊尔马兹取得进球', '中场：姆比亚', '本轮比赛', '姆比亚在与权健中场核心维特塞尔的对抗中丝毫不落下风', '最后时刻还用一记制胜球助华夏幸福继续取胜', '张晓彬', '出任后腰的张晓彬做出5次抢断、4次拦截', '传球成功率超过90%', '他在中场全面的表现也在赛后得到了好评', '多奇卡尔', '此役', '多奇卡尔先是造成了对手的乌龙球', '随后又巧妙助攻哈维尔破门锁定胜局', '可以说是建业取胜的最大功臣之一', '前锋：曹赟定', '全场完成4次射门', '并送出3次威胁传球', '此外还有惊人的8次突破和12次传中', '几乎凭借一己之力搅的力帆禁区天翻地覆', '卡尔德克', '跟曹赟定一样', '本场比赛都是一传一射', '而且卡尔德克的进攻桥头堡作用非常关键', '塔尔德利', '继上次出场梅开二度后', '塔尔德利又在本轮独中两元', '而且比赛中', '“塔神”5次射门全部射正', '向前传球居多的他', '传球成功率竟然高达86%', '毫无疑问', '本赛季脱胎换骨的塔神已经成为了鲁能进攻中的大腿', '此外', '河南建业的新任主帅亚森被评选为本轮最佳教练', '上任不过三天'

'广州恒大战胜贵州恒丰智诚，豪取联赛九连胜的同时继续领跑积分榜，队内的曾诚与张琳芃也入选了本轮中超的最佳阵容。门将：曾诚。尽管在国家队表现不佳，但回到俱乐部的曾诚仅用了一场比赛就为自己实现了正名。全场5次扑救则次次关键，而且，曾诚还扑出了耶拉维奇可能扳平比分的点球，力助恒大豪取九连胜。而且比赛中，“塔神”5次射门全部射正，向前传球居多的他，传球成功率竟然高达86%，毫无疑问，本赛季脱胎换骨的塔神已经成为了鲁能进攻中的大腿。此外，河南建业的新任主帅亚森被评选为本轮最佳教练。上任不过三天，亚森就帮助河南建业拿到了本赛季的第二场胜利，换帅如换刀，'